# Text Classification Using Embeddings
This notebook shows how to build a classifiers using Cohere's embeddings.
<img src="https://github.com/cohere-ai/notebooks/raw/main/notebooks/images/simple-classifier-embeddings.png"
style="width:100%; max-width:600px"
alt="first we embed the text in the dataset, then we use that to train a classifier"/>

The example classification task here will be sentiment analysis of film reviews. We'll train a simple classifier to detect whether a film review is negative (class 0) or positive (class 1).

We'll go through the following steps:

1. Get the dataset
2. Get the embeddings of the reviews (for both the training set and the test set).
3. Train a classifier using the training set
4. Evaluate the performance of the classifier on the testing set

In [ ]:
# Let's first install Cohere's python SDK
!pip install cohere

## 1. Get the dataset

In [ ]:
import pandas as pd
# Get the SST2 training and test sets
df_train = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
df_test = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/test.tsv', delimiter='\t', header=None)

In [ ]:
# Let's glance at the dataset
df_train.head()
print(f"Review #1 text: {df_train.iloc[0, 0]}")
print(f"Review #1 class: {df_train.iloc[0, 1]}")
print(f"Review #2 text: {df_train.iloc[1, 0]}")
print(f"Review #2 class: {df_train.iloc[1, 1]}")

We'll only use a subset of the training and testing datasets in this example. We'll only use 100 examples since this is a toy example. You'll want to increase the number to get better performance and evaluation. 

In [ ]:
n_train_samples = 100 # Increase for better performance (e.g. 500)
n_test_samples = 100 # increase for better evaluation (e.g. 500)

# Sample from the dataset
train = df_train.sample(n_train_samples)
test = df_test.sample(n_test_samples)

sentences_train = list(train.iloc[:,0].values)
sentences_test = list(test.iloc[:,0].values)

labels_train  = list(train.iloc[:,1].values)
labels_test  = list(test.iloc[:,1].values)


## 2. Get the embeddings of the reviews
We're now ready to retrieve the embeddings from the API

In [ ]:
# import cohere, and start a client session
import cohere
co = cohere.Client("") # ADD YOUR API KEY HERE

# embed sentences from both train and test set on small-20211115
embeddings_train = co.embed(model='small-20211115', texts=sentences_train).embeddings
embeddings_test = co.embed(model='small-20211115', texts=sentences_test).embeddings

We now have two sets of embeddings, `embeddings_train` contains the embeddings of the training  sentences while `embeddings_test` contains the embeddings of the testing sentences.

Curious what an embedding looks like? we can print it:

In [ ]:
print(f"Review text: {sentences_train[0]}")
print(f"Embedding vector: {embeddings_train[0][:10]}")

## 3. Train a classifier using the training set
Now that we have the embedding we can train our classifier. We'll use an SVM from sklearn.

In [ ]:
# import support vector machine code
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


# initialize the support vector machine, with class_weight='balanced' because 
# our training set has roughly an equal amount of positive and negative 
# sentiment sentences
svm_classifier = make_pipeline(StandardScaler(), SVC(class_weight='balanced')) 

# fit the support vector machine
svm_classifier.fit(embeddings_train, labels_train)


## 4. Evaluate the performance of the classifier on the testing set

In [ ]:
# get the score from the test set, and print it out to screen!
score = svm_classifier.score(embeddings_test, labels_test)
print(f"Validation accuracy on small-20211115 is {100*score}%!")

This was a small scale example, meant as a proof of concept, and designed to illustrate how you can build a custom
classifier quickly using a small amount of labelled data and Cohere's embeddings. 